# MRmR - regression and classification

Maximal relevance minimal redundancy feature selection is, theoretically, a subset of the all relevant feature selection.



In [1]:
# from IPython.core.display import display, HTML
# display(HTML("<style>.container { width:95% !important; }</style>"))
import gc
import arfs
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sys import getsizeof, path
from arfs.utils import load_data
from arfs.feature_selection import MinRedundancyMaxRelevance
from arfs.preprocessing import TreeDiscretizer

plt.style.use("fivethirtyeight")
rng = np.random.RandomState(seed=42)

import warnings

warnings.filterwarnings("ignore")

In [2]:
print(f"Run with ARFS {arfs.__version__}")

Run with ARFS 2.3.1


In [3]:
gc.enable()
gc.collect()

0

## Simple Usage
In the following examples, I'll use a classical data set to which I added random predictors (numerical and categorical). A maximal relevance minimal redonduncy method should discard them.
In the unit tests, you'll find examples using artifical data with genuine (correlated and non-linear) predictors and with some random/noise columns.

## MRmr

In [4]:
boston = load_data(name="Boston")
X, y = boston.data, boston.target
y.name = "target"

In [5]:
y.head()

0    24.0
1    21.6
2    34.7
3    33.4
4    36.2
Name: target, dtype: float64

In [6]:
X.dtypes

CRIM             float64
ZN               float64
INDUS            float64
CHAS            category
NOX              float64
RM               float64
AGE              float64
DIS              float64
RAD             category
TAX              float64
PTRATIO          float64
B                float64
LSTAT            float64
random_num1      float64
random_num2        int32
random_cat      category
random_cat_2    category
genuine_num      float64
dtype: object

In [7]:
X.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,random_num1,random_num2,random_cat,random_cat_2,genuine_num
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,0.496714,0,cat_3517,Platist,7.080332
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,-0.138264,0,cat_2397,MarkZ,5.245384
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,0.647689,0,cat_3735,Dracula,6.375795
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,1.523030,0,cat_2870,Bejita,6.725118
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,-0.234153,4,cat_1160,Variance,7.867781


In [8]:
fs_mrmr = MinRedundancyMaxRelevance(
    n_features_to_select=5,
    relevance_func=None,
    redundancy_func=None,
    task="regression",  # "classification",
    denominator_func=np.mean,
    only_same_domain=False,
    return_scores=False,
    show_progress=True,
    n_jobs=-1,
)

# fs_mrmr.fit(X=X, y=y.astype(str), sample_weight=None)
fs_mrmr.fit(X=X, y=y, sample_weight=None)

  0%|          | 0/5 [00:00<?, ?it/s]

/home/bsatom/Documents/arfs/src/arfs/association.py:225: RuntimeWarning: Rounded U = 1.0 to 1. This is probably due to floating point precision issues.
  warnings.warn(
/home/bsatom/Documents/arfs/src/arfs/association.py:225: RuntimeWarning: Rounded U = 1.0 to 1. This is probably due to floating point precision issues.
  warnings.warn(


MinRedundancyMaxRelevance(n_features_to_select=5, n_jobs=-1,
                          redundancy_func=functools.partial(<function association_series at 0x764928fd4680>, n_jobs=-1, normalize=True),
                          relevance_func=functools.partial(<function f_stat_regression_parallel at 0x764928fd4b80>, n_jobs=-1))

In [9]:
X_trans = fs_mrmr.transform(X)
X_trans.head()

,LSTAT,genuine_num,CHAS,RAD,RM
0,4.98,7.080332,0.0,1.0,6.575
1,9.14,5.245384,0.0,2.0,6.421
2,4.03,6.375795,0.0,2.0,7.185
3,2.94,6.725118,0.0,3.0,6.998
4,5.33,7.867781,0.0,3.0,7.147


Using a single job, avoiding the overhead of starting multiple processes (for moderate size data)

In [10]:
fs_mrmr = MinRedundancyMaxRelevance(
    n_features_to_select=5,
    relevance_func=None,
    redundancy_func=None,
    task="regression",  # "classification",
    denominator_func=np.mean,
    only_same_domain=False,
    return_scores=False,
    show_progress=True,
    n_jobs=1,
)

fs_mrmr.fit(X=X, y=y, sample_weight=None)

  0%|          | 0/5 [00:00<?, ?it/s]

MinRedundancyMaxRelevance(n_features_to_select=5,
                          redundancy_func=functools.partial(<function association_series at 0x764928fd4680>, n_jobs=1, normalize=True),
                          relevance_func=functools.partial(<function f_stat_regression_parallel at 0x764928fd4b80>, n_jobs=1))

In [11]:
fs_mrmr.feature_names_in_

array(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD',
       'TAX', 'PTRATIO', 'B', 'LSTAT', 'random_num1', 'random_num2',
       'random_cat', 'random_cat_2', 'genuine_num'], dtype=object)

In [12]:
fs_mrmr.support_

array([False, False, False,  True, False,  True, False, False,  True,
       False, False, False,  True, False, False, False, False,  True])

In [13]:
fs_mrmr.get_feature_names_out()

array(['CHAS', 'RM', 'RAD', 'LSTAT', 'genuine_num'], dtype=object)

In [14]:
fs_mrmr.ranking_

,mrmr,relevance,redundancy
LSTAT,inf,3.026693,0.000000
genuine_num,1131.029947,1.131030,0.001000
CHAS,1.787263,0.731266,0.409154
RAD,2.262116,0.990593,0.437905
RM,0.571875,0.145600,0.254601


## Classification 

In [15]:
titanic = load_data(name="Titanic")
X, y = titanic.data, titanic.target

# y = y.astype('category')
y.name = "target"

In [16]:
X.head()

,pclass,sex,embarked,random_cat,is_alone,title,age,family_size,fare,random_num
0,1,female,S,Fry,1,Mrs,29.0000,0.0,211.3375,0.496714
1,1,male,S,Bender,0,Master,0.9167,3.0,151.5500,-0.138264
2,1,female,S,Thanos,0,Mrs,2.0000,3.0,151.5500,0.647689
3,1,male,S,Morty,0,Mr,30.0000,3.0,151.5500,1.523030
4,1,female,S,Morty,0,Mrs,25.0000,3.0,151.5500,-0.234153


In [17]:
X.dtypes

pclass          object
sex             object
embarked        object
random_cat      object
is_alone        object
title           object
age            float64
family_size    float64
fare           float64
random_num     float64
dtype: object

In [18]:
y

0       1
1       1
2       0
3       0
4       0
       ..
1304    0
1305    0
1306    0
1307    0
1308    0
Name: target, Length: 1309, dtype: category
Categories (2, object): ['0', '1']

In [19]:
fs_mrmr = MinRedundancyMaxRelevance(
    n_features_to_select=5,
    relevance_func=None,
    redundancy_func=None,
    task="classification",
    denominator_func=np.mean,
    only_same_domain=False,
    return_scores=False,
    show_progress=True,
    n_jobs=1,
)

# fs_mrmr.fit(X=X, y=y.astype(str), sample_weight=None)
fs_mrmr.fit(X=X, y=y, sample_weight=None)

  0%|          | 0/5 [00:00<?, ?it/s]

MinRedundancyMaxRelevance(n_features_to_select=5,
                          redundancy_func=functools.partial(<function association_series at 0x764928fd4680>, n_jobs=1, normalize=True),
                          relevance_func=functools.partial(<function f_stat_classification_parallel at 0x764928fd4e00>, n_jobs=1),
                          task='classification')

In [20]:
fs_mrmr.feature_names_in_

array(['pclass', 'sex', 'embarked', 'random_cat', 'is_alone', 'title',
       'age', 'family_size', 'fare', 'random_num'], dtype=object)

In [21]:
fs_mrmr.support_

array([False,  True, False, False,  True,  True, False,  True,  True,
       False])

In [22]:
fs_mrmr.get_feature_names_out()

array(['sex', 'is_alone', 'title', 'family_size', 'fare'], dtype=object)

In [23]:
fs_mrmr.ranking_

,mrmr,relevance,redundancy
sex,inf,1.740237,0.000000
fare,8.112641,1.499352,0.184817
title,1.483820,0.694140,0.467806
family_size,-1.318770,-0.516471,0.391631
is_alone,-2.063718,-0.639133,0.309700


## Pipeline

Integration as a step of a feature selection pipeline

In [24]:
from sklearn.pipeline import Pipeline
from arfs.feature_selection import (
    MissingValueThreshold,
    UniqueValuesThreshold,
    make_fs_summary,
)

titanic = load_data(name="Titanic")
X, y = titanic.data, titanic.target

# y = y.astype('category')
y.name = "target"

fs_mrmr = MinRedundancyMaxRelevance(
    n_features_to_select=5,
    relevance_func=None,
    redundancy_func=None,
    task="classification",
    denominator_func=np.mean,
    only_same_domain=False,
    return_scores=False,
    show_progress=True,
    n_jobs=1,
)

mrmr_fs_pipeline = Pipeline(
    [
        ("missing", MissingValueThreshold(threshold=0.05)),
        ("unique", UniqueValuesThreshold(threshold=1)),
        ("mrmr", fs_mrmr),
    ]
)

X_trans = mrmr_fs_pipeline.fit(X=X, y=y).transform(X=X)
#   collinearity__sample_weight=w,
#   lowimp__sample_weight=w)
X_trans.head()

  0%|          | 0/5 [00:00<?, ?it/s]

,sex,fare,title,family_size,is_alone
0,female,211.3375,Mrs,0.0,1
1,male,151.5500,Master,3.0,0
2,female,151.5500,Mrs,3.0,0
3,male,151.5500,Mr,3.0,0
4,female,151.5500,Mrs,3.0,0


In [25]:
make_fs_summary(mrmr_fs_pipeline)

,predictor,missing,unique,mrmr
0,pclass,1,1,0
1,sex,1,1,1
2,embarked,1,1,0
3,random_cat,1,1,0
4,is_alone,1,1,1
5,title,1,1,1
6,age,1,1,0
7,family_size,1,1,1
8,fare,1,1,1
9,random_num,1,1,0


In [26]:
mrmr_fs_pipeline.named_steps["mrmr"].get_feature_names_out()

array(['sex', 'is_alone', 'title', 'family_size', 'fare'], dtype=object)

## Does discretization help?

We can use `TreeDiscretizer` to discretize and auto-group the predictors (whatever if numeric or not). Does that improves the MRmr output?

In [27]:
# main parameter controlling how agressive will be the auto-grouping
lgb_params = {"min_split_gain": 0.05}
# instanciate the discretizer
disc = TreeDiscretizer(bin_features="all", n_bins=10, boost_params=lgb_params)

titanic = load_data(name="Titanic")
X, y = titanic.data, titanic.target

# y = y.astype('category')
y.name = "target"
y = y.astype("int")

fs_mrmr = MinRedundancyMaxRelevance(
    n_features_to_select=5,
    relevance_func=None,
    redundancy_func=None,
    task="classification",
    denominator_func=np.mean,
    only_same_domain=False,
    return_scores=False,
    show_progress=True,
    n_jobs=1,
)

mrmr_fs_pipeline = Pipeline(
    [
        ("missing", MissingValueThreshold(threshold=0.05)),
        ("unique", UniqueValuesThreshold(threshold=1)),
        ("discretizer", disc),
        ("mrmr", fs_mrmr),
    ]
)

X_trans = mrmr_fs_pipeline.fit(X=X, y=y).transform(X=X)
#   collinearity__sample_weight=w,
#   lowimp__sample_weight=w)
X_trans.head()

  0%|          | 0/5 [00:00<?, ?it/s]

,sex,title,is_alone,pclass,embarked
0,female,Mrs,1,1,S / missing / Q
1,male,Master,0,1,S / missing / Q
2,female,Mrs,0,1,S / missing / Q
3,male,Mr,0,1,S / missing / Q
4,female,Mrs,0,1,S / missing / Q


In [28]:
make_fs_summary(mrmr_fs_pipeline)

,predictor,missing,unique,discretizer,mrmr
0,pclass,1,1,nan,1
1,sex,1,1,nan,1
2,embarked,1,1,nan,1
3,random_cat,1,1,nan,0
4,is_alone,1,1,nan,1
5,title,1,1,nan,1
6,age,1,1,nan,0
7,family_size,1,1,nan,0
8,fare,1,1,nan,0
9,random_num,1,1,nan,0
